# Легкий способ парсинга новостных статей на Python

В современном цифровом мире нас засыпают бесконечным потоком информации.
Мы постоянно прокручиваем социальные сети и 24 часа в сутки имеем доступ к новостным каналам.
Таким образом, существует множество новостей, которые нам надо знать и мы должны быть в состоянии их все быстро переварить.

Итак, давайте разберем упражнение по сжатию новостных статей до размера, более удобного для их восприятия.

Мы спарсим примерную статью, используя библиотеки request и BeautifulSoup, а затем сформируем ее краткое изложение при помощи великолепной библиотеки gensim.

In [10]:
# Импорт необходимых библиотек
# Pandas & Numpy
# import pandas as pd
# import numpy as np
# import sys

import requests
from bs4 import BeautifulSoup
from gensim.summarization import summarize

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject

In [8]:
# MISTAKE !!!!
# ModuleNotFoundError: No module named 'numpy.random.bit_generator'

# Запуск следующей строки заставил переустановить numpyпакет.
# Поскольку пакет был каким-то образом поврежден, это было исправлено.
# conda install numpy --force-reinstall

# Или можно попробовать так:
# pip3 install scipy==1.7.1 numpy==1.18.5 scikit-learn==0.24.2 --no-cache-dir --no-binary :all:

!pip install numpy --force-reinstall

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [9]:
# Установка библиотеки для более удобного для их восприятия текстовых документов и статей
# !pip install gensim

#### Статья по которой воспроизводится данный урок лежит по ссылке:

https://pythonist.ru/legkij-sposob-parsinga-novostnyh-statej-na-python/

In [ ]:
# Теперь выберем интересную статью:
# Получаем текст страницы
url = 'https://www.npr.org/2019/07/10/740387601/university-of-texas-austin-promises-free-tuition-for-low-income-students-in-2020'
page = requests.get(url).text
# print(page)

## Вебскрейпинг
### Теперь приступим непосредственно к вебскрейпингу!

In [ ]:
# Сначала мы превратим содержимое страницы в объект BeautifulSoup, что позволит нам анализировать HTML-теги.
# Turn page into BeautifulSoup object to access HTML tags
soup = BeautifulSoup(page)

Затем нам нужно выяснить, какие HTML-теги содержат заголовок и основной текст статьи.
В качестве отличного учебника по HTML можно использовать сайт HTML.com.

[python_ad_block]
Чтобы это сделать, мы будем использовать инструменты разработчика Google Chrome.
Откроем статью в новой вкладке, кликнем по ней правой кнопкой мыши и в выпавшем меню выберем пункт Inspect (Просмотр кода).
Это вызовет DevTools (инструменты разработчика) в панели справа:

Чтобы найти все HTML-теги, соответствующие всему, что вы видите на странице, нажмите на небольшую кнопочку наверху.
На картинке она отмечена синей стрелкой.

Теперь наводим указатель мыши на фрагмент страницы, который мы хотим исследовать.
В данном случае это заголовок и основной текст статьи. И мы видим, какие теги отвечают за формат данного текста.

In [ ]:
# Заголовок статьи окружен c двух сторон тегом <h1>. Загрузим этот фрагмент следующим образом:

# Get headline
headline = soup.find('h1').get_text()

In [ ]:
# Основной текст статьи окружен тегами <p>. На этот раз нам нужно будет найти все такие теги,
# содержащиеся на странице, поскольку в них заключен каждый абзац статьи.

# Get text from all <p> tags.
p_tags = soup.find_all('p’)
# Get the text from each of the “p” tags and strip surrounding whitespace.
p_tags_text = [tag.get_text().strip() for tag in p_tags]

Если вы изучите текст, который мы сохранили в переменную p_tags_text, то заметите, что есть некоторые фрагменты текста, не относящиеся к основной статье, например имя автора и некоторые подписи к изображениям.
Они также заключены в теги <p>, поэтому и оказались здесь.
Чтобы очистить этот текст от фрагментов, которые не являются частью основной статьи, мы в качестве фильтров используем несколько представлений списков.

В этой статье подписи к изображениям содержат символ новой строки \n.
Поскольку мы знаем, что в настоящих предложениях в статье не бывает случайных переносов строк, мы можем смело отказаться от таких фрагментов.
Точно так же мы можем отбрасывать фрагменты текста, не содержащие точки, поскольку мы знаем, что любое правильное предложение в статье будет содержать точку.
Так мы сможем отбросить имя автора и некоторые другие ненужные нам вещи.

In [ ]:
# 
# Filter out sentences that contain newline characters '\n' or don't contain periods.
sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
# Combine list items into string.
article = ' '.join(sentence_list)

## Резюме статьи

Теперь, имея полный текст статьи, мы хотим получить его резюме (краткий пересказ содержания статьи).

Gensim — отличный пакет Python для большого количества задач нейролингвистического программирования (НЛП).
Он включает в себя довольно надежную функцию резюмирования, которой достаточно легко пользоваться.
Она реализует разновидность алгоритма TextRank.

In [ ]:
# Для использования этой функции нам нужна лишь одна строчка кода:

summary = summarize(article_text, ratio=0.3)

## Вот и все, мы сделали, что хотели!

### Мы извлекли заголовок статьи и получили краткое изложение ее содержания.

#### Теперь вы можете понять суть статьи примерно в три раза быстрее и сэкономить время для других дел. 

In [ ]:
# 


In [ ]:
# 


# Скрейпинг сайтов с помощью библиотек Beautifulsoup и Requests на Python

### Делаем совместно с каналом "Важные Истории" на YouTube и его ведущей Алесей 

In [ ]:
# https://www.youtube.com/watch?v=0ws5tsRBgL8

In [5]:
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеку для парсинга сайтов !!!!!!!!!!!!!
import requests
import csv

In [29]:
# Переменная со ссылкой на сайт
# url = 'https://www.kinopoisk.ru/lists/movies/top250/'

url = 'https://www.kinopoisk.ru/lists/categories/movies/1/'
# https://www.kinopoisk.ru/lists/movies/top250/

In [30]:
# Создаём запрос
response = requests.get(url)
# response.text

In [31]:
# Объект SOUP
soup = BeautifulSoup(response.text, 'lxml')
soup

<html><body></body><script nonce="9ea8db73cbda0fdae8857745c46f37cf">var it = {"host":"https:\u002F\u002Fsso.kinopoisk.ru\u002Finstall?uuid=7c2d03d6-4a04-4dea-bd95-d646825e7a5c","retpath":"https:\u002F\u002Fwww.kinopoisk.ru\u002Flists\u002Fcategories\u002Fmovies\u002F1\u002F"};(function() { var form = document.createElement('form'); var element1 = document.createElement('input'); var element2 = document.createElement('input'); element1.name = 'retpath'; element1.type = 'hidden'; element1.value = it.retpath; form.appendChild(element1); element2.name = 'container'; element2.type = 'hidden'; element2.value = '1732034935.10150276.79Iw7_Jy9u7S-9yV.6W04ceONJ4nImUYKqkqzI7h3fSiUczhG9dp9Ybjt4HcF3z6pPvYKv8pETZOVx-fngz0XJqLmOEDsYx_P5fJAwy2jLQ39oLovuIXc1f2r_sOQ5WgZHm36k-CJ0HHXmAHryWZDhyLBrW5kHLN-mGG1H4vv4ZVJRCcLOzo7QocXqKSPblLQt9QvT1hM21ieZ2NsS7QSM5gUJo7AEwfaI91CrXxpwtNlAhNfOhLXMdtLRAHA2NivGOibMRAZL69wCwKHRba1ksfD5d6ehQlLAL_kpLhuC9A-GZzasFL62EuIxOp2dBoKClN0pnsuaLz9CrFjh0Oo_eh2spqZHeHgYKzxAOY6lpkSoi

In [32]:
# Определяем DIV-ы с нужной инфой (заголовки статей)
# film = soup.find('div', class_ = 'styles_root__ti07r')
film = soup.find('div', class_ = 'styles_meta__M_kDW')
print(film)

None


# Простой парсинг сайтов на Python | requests, BeautifulSoup, csv

### Делаем совместно с каналом Андрея Андриевского

In [34]:
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеку для парсинга сайтов !!!!!!!!!!!!!
import requests
import csv

In [ ]:
# Создаём переменные-КОНСТАНТЫ
# HOST = 'https://www.kinopoisk.ru/'
# URL = 'https://www.kinopoisk.ru/lists/movies/top250/'
# HEADERS = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
# }

In [76]:
# Создаём переменные-КОНСТАНТЫ
HOST = 'https://vc.ru'
URL = 'https://vc.ru/popular'
HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}
CSV = 'catalog.csv'

#### Парсинг осуществляется при помощи нескольких функции, вызываемых главной функцией (не ООП !!!) 

In [46]:
# 1 Функция - обращение к вебстранице для получения кода HTML:
def get_html(url, params=''):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

In [51]:
# 
result = get_html(URL)
print(result)

# <Response [200]> - это значит, что ошибки нет и в переменной содержится код страницы

<Response [200]>


In [72]:
# 2 Функция, которая будет получать нужную для наших задач информацию 
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
#     soup = BeautifulSoup(html, 'lxml')
    articles = soup.find_all('div', class_ = 'content content--short')
    catalog = []
#     print(item)
    
    for article in articles:
        catalog.append({
            'title': article.find('div', class_ = 'content-title').get_text(),
            'text': article.find('div', class_ = 'block-text').get_text(),
            'link': HOST + article.find('a', class_ = 'content__link')['href']
        })
        
    return catalog

In [75]:
# 
content = get_content(result.text)
content

[{'title': 'Официальный курс доллара превысил 100 рублей ',
  'text': 'Курс на 20 ноября 2024 года — 100,03 рубля.',
  'link': 'https://vc.ru/money/1662348-oficialnyi-kurs-dollara-prevysil-100-rublei'},
 {'title': 'Jaguar представил новый логотип — без ягуара в нём ',
  'text': 'Но его изображение останется одним из элементов фирменного стиля.',
  'link': 'https://vc.ru/design/1662196-jaguar-predstavil-novyi-logotip-bez-yaguara-v-nem'},
 {'title': 'Производитель «умных» колец Oura привлёк $75 млн при оценке в $5 млрд ',
  'text': 'Оценка выросла в два раза с 2022 года. ',
  'link': 'https://vc.ru/invest/1662441-proizvoditel-umnyh-kolec-oura-privlek-75-mln-pri-ocenke-v-5-mlrd'},
 {'title': '«Предприниматель должен уметь работать в любых условиях»: основатель «Красное & Белое» — об онлайн-продаже алкоголя, параллельном импорте и возможном IPO ',
  'text': 'Тезисы из беседы Сергея Студенникова с РБК.',
  'link': 'https://vc.ru/retail/1656031-predprinimatel-dolzhen-umet-rabotat-v-lyubyh-us

In [ ]:
# Функция для записи результатов парсинга в файл
def save_doc(articles, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['TITLE', 'TEXT', 'LINK'])
        for item in items:
            writer.writerow([
                item['title'],
                item['text'],
                item['link']
            ])

In [ ]:
# Функция для обработки и запуска всех функций и определения количества страниц для парсинга:
def parser():
    PAGENATION =input('Сколько страниц вы хотите запарсить? ')
    PAGENATION = int(PAGENATION.strip())
    html = get_html(URL)
    
    if html.status == 200:
        catalog = []
        for page in range(1, PAGENATION):
            print(f'Парсим страницу: {page}')
            html = get_html(URL, params={'page': page})
            catalog.extend(get_content(html.text))
        print('PARSING ENDED')
        print(catalog)
        save_doc(catalog, CSV)
    else:
        print('ERROR !!!!')

In [ ]:
## 
parser()

In [ ]:
# 


In [ ]:
## 


In [ ]:
# 


In [ ]:
# 


In [ ]:
## 


In [ ]:
# 


In [ ]:
# 


In [ ]:
## 


In [ ]:
# 


In [ ]:
# 


In [ ]:
## 
